In [154]:
import numpy as np
results_task1 = np.load('data/stateValuesTask1.npy')
grid_world = np.load('data/gridworld.npy')

In [157]:
grid_world

array([[ 10.,  10.,  10.,  -1.,  -1.,  -1.,  -1.,  10.,  10.],
       [ -1.,  -1.,  -1.,  -1.,  -1., -20.,  -1., -20.,  10.],
       [ -1., -20., -20., -20., -20., -20.,  -1., -20.,  -1.],
       [ -1., -20.,  10.,  10.,  10.,  10., -20.,  -1., 100.],
       [ -1.,  -1.,  10., -20., -20., -20.,  -1.,  -1.,  -1.],
       [ -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1., -20.,  -1.],
       [ -1., -20., -20., -20., -20., -20., -20.,  -1.,  -1.],
       [ -1.,  10.,  10.,  10.,  10.,  10.,  10.,  -1., -20.],
       [ -1.,  10.,  10.,  10., -20.,  10.,  10.,  -1., -20.]])

In [158]:
results_task1

array([[ -55.72285158,  -60.28813092,  -76.21943677,  -85.94239006,
         -95.17214143, -102.06460192, -109.14849891, -128.03672486,
        -154.35820755],
       [ -70.24180495,  -71.03860383,  -82.4266548 ,  -97.04877034,
        -113.45253199,    0.        , -138.83622028,    0.        ,
        -140.1279616 ],
       [-114.35635123,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        , -156.41838188,    0.        ,
         -71.58053207],
       [-113.49056343,    0.        ,  -67.69447722,  -82.24055051,
        -104.22849427, -131.61383646,    0.        ,   67.97543479,
           0.        ],
       [ -66.87372941,  -61.70095357,  -77.43127146,    0.        ,
           0.        ,    0.        ,  -51.56857317,  -41.72612892,
         -63.11723718],
       [ -64.4840581 ,  -63.86563014,  -69.53147116,  -77.8922909 ,
         -83.19180027,  -88.82103118,  -95.51685587,    0.        ,
        -145.95964503],
       [ -81.41499169,    0.        ,   

In [252]:
goal = {'x':8,'y':3}

finished = False

def is_outside(state):
    return ((state['y'] < 0 or state['y'] >= grid_world.shape[0]) 
              or (state['x'] < 0 or state['x'] >= grid_world.shape[1]))

def terminal_state(state):
    # Entered a goal state
    if state == goal:
        return True
    # Exeeded boundary limits
    elif is_outside(state):
        return True
    # Entered a cell X
    elif grid_world[state['y'], state['x']] == -20:
        return True

    # Not a terminal state
    else:
        return False

def get_reward(state):
    if is_outside(state):
        return -30;
    else:
        return grid_world[state['y'], state['x']]

def make_move(state, action):
    state_c = state.copy()
    
    if action == 'l':
        state_c['x']  -= 1
    elif action == 'r':
        state_c['x']  += 1
    elif action == 'u':
        state_c['y']  -= 1
    elif action == 'd':
        state_c['y']  += 1
        
    if not terminal_state(state_c):
        state = state_c
    
    return get_reward(state_c)

    
def compute_V(state, count, disc=0.9):
    if count == 6 or terminal_state(state):
        return 0
    
    return_V = 0
    count += 1

    # Right + random action
    state_r = state.copy()
    reward = make_move(state_r, 'r')
    return_V += (0.5 + 0.125) * (reward + disc**count * compute_V(state_r, count))
    
    # Random left action
    state_l = state.copy()
    reward = make_move(state_l, 'l')
    return_V += 0.125 * (reward + disc**count * compute_V(state_l, count))
    
    # Random up action
    state_u = state.copy()
    reward = make_move(state_u, 'u')
    return_V += 0.125 * (reward + disc**count * compute_V(state_u, count))
            
    # Random down action
    state_d = state.copy()
    reward = make_move(state_d, 'd')
    return_V += 0.125 * (reward + disc**count * compute_V(state_d, count))
        
    return return_V

In [253]:
state = {'x':8,'y':2}
compute_V(state, 0)

-27.862579291459888

In [248]:
np.set_printoptions(suppress=True)
grid_task1 = np.zeros_like(grid_world)

for y in range(grid_world.shape[0]):
    for x in range(grid_world.shape[1]):
        grid_task1[y,x] = compute_V({'y': y, 'x': x}, 0)
    


In [249]:
grid_task1import numpy as np

results_task1 = np.load('data/stateValuesTask1.npy')

grid_world = np.load('data/gridworld.npy')

grid_world

array([[ 10.,  10.,  10.,  -1.,  -1.,  -1.,  -1.,  10.,  10.],
       [ -1.,  -1.,  -1.,  -1.,  -1., -20.,  -1., -20.,  10.],
       [ -1., -20., -20., -20., -20., -20.,  -1., -20.,  -1.],
       [ -1., -20.,  10.,  10.,  10.,  10., -20.,  -1., 100.],
       [ -1.,  -1.,  10., -20., -20., -20.,  -1.,  -1.,  -1.],
       [ -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1., -20.,  -1.],
       [ -1., -20., -20., -20., -20., -20., -20.,  -1.,  -1.],
       [ -1.,  10.,  10.,  10.,  10.,  10.,  10.,  -1., -20.],
       [ -1.,  10.,  10.,  10., -20.,  10.,  10.,  -1., -20.]])

results_task1

array([[ -55.72285158,  -60.28813092,  -76.21943677,  -85.94239006,
         -95.17214143, -102.06460192, -109.14849891, -128.03672486,
        -154.35820755],
       [ -70.24180495,  -71.03860383,  -82.4266548 ,  -97.04877034,
        -113.45253199,    0.        , -138.83622028,    0.        ,
        -140.1279616 ],
       [-114.35635123,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        , -156.41838188,    0.        ,
         -71.58053207],
       [-113.49056343,    0.        ,  -67.69447722,  -82.24055051,
        -104.22849427, -131.61383646,    0.        ,   67.97543479,
           0.        ],
       [ -66.87372941,  -61.70095357,  -77.43127146,    0.        ,
           0.        ,    0.        ,  -51.56857317,  -41.72612892,
         -63.11723718],
       [ -64.4840581 ,  -63.86563014,  -69.53147116,  -77.8922909 ,
         -83.19180027,  -88.82103118,  -95.51685587,    0.        ,
        -145.95964503],
       [ -81.41499169,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        , -146.68306869,
        -167.4479631 ],
       [ -24.07747643,  -19.83472912,  -28.50505501,  -42.3162275 ,
         -57.39664499,  -75.06625414, -100.9614636 , -122.15419146,
           0.        ],
       [ -29.38658069,  -33.23429959,  -49.04161122,  -73.92283119,
           0.        ,  -85.56598202, -105.10495463, -125.19771008,
           0.        ]])

goal = {'x':8,'y':3}

​

finished = False

​

def is_outside(state):

    return ((state['y'] < 0 or state['y'] >= grid_world.shape[0]) 

              or (state['x'] < 0 or state['x'] >= grid_world.shape[1]))

​

def terminal_state(state):

    # Entered a goal state

    if state == goal:

        return True

    # Exeeded boundary limits

    elif is_outside(state):

        return True

    # Entered a cell X

    elif grid_world[state['y'], state['x']] == -20:

        return True

​

    # Not a terminal state

    else:

        return False

​

def get_reward(state):

    if is_outside(state):

        return -30;

    else:

        return grid_world[state['y'], state['x']]

​

def make_move(state, action):

    state_c = state.copy()

    

    if action == 'l':

        state_c['x']  -= 1

    elif action == 'r':

        state_c['x']  += 1

    elif action == 'u':

        state_c['y']  -= 1

    elif action == 'd':

        state_c['y']  += 1

        

    if not terminal_state(state_c):

        state = state_c

    

    return get_reward(state_c)

​

    

def compute_V(state, count, disc=0.9):

    if count == 6 or terminal_state(state):

        return 0

    

    return_V = 0

    count += 1

​

    # Right + random action

    state_r = state.copy()

    reward = make_move(state_r, 'r')

    return_V += (0.5 + 0.125) * (reward + disc**count * compute_V(state_r, count))

    

    # Random left action

    state_l = state.copy()

    reward = make_move(state_l, 'l')

    return_V += 0.125 * (reward + disc**count * compute_V(state_l, count))

    

    # Random up action

    state_u = state.copy()

    reward = make_move(state_u, 'u')

    return_V += 0.125 * (reward + disc**count * compute_V(state_u, count))

            

    # Random down action

    state_d = state.copy()

    reward = make_move(state_d, 'd')

    return_V += 0.125 * (reward + disc**count * compute_V(state_d, count))

        

    return return_V

state = {'x':8,'y':2}

compute_V(state, 0)

-27.862579291459888

np.set_printoptions(suppress=True)

grid_task1 = np.zeros_like(grid_world)

​

for y in range(grid_world.shape[0]):

    for x in range(grid_world.shape[1]):

        grid_task1[y,x] = compute_V({'y': y, 'x': x}, 0)

    

​

grid_task1

array([[ -5.10813954,  13.46691332, -12.07378436, -12.07378436,
        -17.1819239 , -26.00507401,   8.35877379,  -0.46437632,
        -74.30021144],
       [-12.07378436,  -7.43002114,  -7.43002114, -12.53816068,

array([[ -5.10813954,  13.46691332, -12.07378436, -12.07378436,
        -17.1819239 , -26.00507401,   8.35877379,  -0.46437632,
        -74.30021144],
       [-12.07378436,  -7.43002114,  -7.43002114, -12.53816068,
        -56.65391123,   0.        , -56.65391123,   0.        ,
        -74.76458777],
       [-61.29767444,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        , -65.47706133,   0.        ,
        -27.86257929],
       [-61.29767444,   0.        ,   9.28752643,   9.28752643,
          9.28752643, -60.3689218 ,   0.        , 213.14873158,
          0.        ],
       [-17.1819239 ,  13.002537  , -42.72262158,   0.        ,
          0.        ,   0.        , -21.36131079, -12.53816068,
        -24.14756872],
       [-17.1819239 , -12.53816068,  -7.43002114, -21.36131079,
        -21.36131079, -21.36131079, -56.65391123,   0.        ,
        -79.8727273 ],
       [-61.29767444,   0.        ,   0.        ,   0.        ,
          0.        ,   0.    